# Step 1: Importing the data

In [22]:
from mlxtend.frequent_patterns import apriori, association_rules

In [23]:
import pandas as pd
import numpy as np

df=pd.read_excel("http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx")

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [24]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [25]:
# Exploring the different regions of transactions 
df.Country.value_counts()

United Kingdom          495478
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon 

# Step 2: Cleaning the data

In [26]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [27]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [30]:
"""
Read about dataset: http://archive.ics.uci.edu/ml/datasets/Online+Retail

It contains transactions about a company that sells unique all occassions giftset.
Invoice No: If this code starts with letter 'c', it indicates a cancellation- We will drop all cancellations for the analysis.
StockCode: Product (item) code. - we will use the item code for groupby
Description: Product (item) name.
Country:the name of the country where each customer resides

"""
df['InvoiceNo']=df['InvoiceNo'].astype('str')
df['StockCode']=df['StockCode'].str.strip()
df['Description']=df['Description'].str.strip()
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [31]:
df.isnull().sum()

InvoiceNo           0
StockCode      487036
Description      1455
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [32]:
"""
Dropping null rows 
Dropping all cancellations
"""

df.dropna(inplace=True)
df=df[~df.InvoiceNo.str.startswith("C")]
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
45,536370,POST,POSTAGE,3,2010-12-01 08:45:00,18.00,12583.0,France


You can see that there are incorrect values like "POST" in the StockCode. StockCode column contains 5-digit integral number uniquely assigned to each distinct product.

In [34]:
#Marked as 1 whereever there is inaccurate value for StockCode
df['StockCode_Check']=df['StockCode'].apply(lambda x : 1 if len(x)<5 else 0)

In [35]:
df=df[df['StockCode_Check']==0]

We will do a similar check for InvoiceNo. InvoiceNo is a 6-digit integral number uniquely assigned to each transaction.

In [40]:
#Marked as 1 whereever there is inaccurate value for InvoiceNo
df['InvoiceNo_Check']=df['InvoiceNo'].apply(lambda x : 1 if len(x)<6 else 0)
df=df[df['InvoiceNo_Check']==0]

# Step 3: Splitting the data according to the region of transaction

In [72]:
basket_UK =pd.pivot_table(df[df['Country']=='United Kingdom'],columns="Description",index="InvoiceNo",values="Quantity",aggfunc=np.sum,
              fill_value=0)

In [73]:
basket_France =pd.pivot_table(df[df['Country']=='France'],columns="Description",index="InvoiceNo",values="Quantity",aggfunc=np.sum,
              fill_value=0)

In [74]:
basket_Germany =pd.pivot_table(df[df['Country']=='Germany'],columns="Description",index="InvoiceNo",values="Quantity",aggfunc=np.sum,
              fill_value=0)

In [75]:
basket_Spain =pd.pivot_table(df[df['Country']=='Spain'],columns="Description",index="InvoiceNo",values="Quantity",aggfunc=np.sum,
              fill_value=0)

# Step 4: Hot encoding the Data

In [76]:
# Defining the hot encoding function to make the data suitable  
# for the concerned libraries 

def hot_encode(x):
    if(x>=1):
        return 1
    if(x<=0):
        return 0

In [77]:
"""
Difference between apply and applymap:

apply(): used on both on a pandas dataframe and series. The function passed as an argument typically works on rows/columns
eg: gfg_df.apply(lambda x:x.sort_values(), axis = 1) 

applymap(): can be used on a pandas dataframe. The function passed as an argument typically works on elements of the dataframe
applymap() is typically used for elementwise operations. 
eg: gfg_df.applymap(str.upper) 

"""
basket_encoded=basket_UK.applymap(hot_encode)
basket_UK=basket_encoded

In [78]:
#ncoding France
basket_encoded=basket_France.applymap(hot_encode)
basket_France=basket_encoded

In [79]:
#encoding Germany
basket_encoded=basket_Germany.applymap(hot_encode)
basket_Germany=basket_encoded

# Step 5: Building the model

## 1. UK

In [88]:
#5% Support means total 5% of transactions in database follow the rule.

frq_items = apriori(basket_UK, min_support = 0.01, use_colnames = True) 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
39,"(SMALL MARSHMALLOWS PINK BOWL, SMALL CHOCOLATE...",(SMALL DOLLY MIX DESIGN ORANGE BOWL),0.012814,0.036733,0.011426,0.891667,24.274588,0.010955,8.891700
40,"(SMALL DOLLY MIX DESIGN ORANGE BOWL, SMALL CHO...",(SMALL MARSHMALLOWS PINK BOWL),0.013454,0.028938,0.011426,0.849206,29.346190,0.011036,6.439677
25,(SMALL CHOCOLATES PINK BOWL),(SMALL DOLLY MIX DESIGN ORANGE BOWL),0.016231,0.036733,0.013454,0.828947,22.567128,0.012858,5.631410
27,(SMALL CHOCOLATES PINK BOWL),(SMALL MARSHMALLOWS PINK BOWL),0.016231,0.028938,0.012814,0.789474,27.281997,0.012344,4.612547
28,(SMALL MARSHMALLOWS PINK BOWL),(SMALL DOLLY MIX DESIGN ORANGE BOWL),0.028938,0.036733,0.022531,0.778598,21.196419,0.021468,4.350758
32,"(JUMBO BAG STRAWBERRY, JUMBO BAG BAROQUE BLAC...",(JUMBO BAG RED RETROSPOT),0.021356,0.154511,0.015910,0.745000,4.821648,0.012611,3.315641
5,(CHILDRENS CUTLERY POLKADOT BLUE),(CHILDRENS CUTLERY POLKADOT PINK),0.017726,0.026268,0.013027,0.734940,27.978499,0.012562,3.673625
3,(BLUE HAPPY BIRTHDAY BUNTING),(PINK HAPPY BIRTHDAY BUNTING),0.032355,0.033422,0.023171,0.716172,21.427946,0.022090,3.405500
43,(SMALL CHOCOLATES PINK BOWL),"(SMALL MARSHMALLOWS PINK BOWL, SMALL DOLLY MIX...",0.016231,0.022531,0.011426,0.703947,31.243920,0.011060,3.301674
6,(CHILDRENS CUTLERY POLKADOT GREEN),(CHILDRENS CUTLERY RETROSPOT RED),0.014522,0.019434,0.010144,0.698529,35.943560,0.009862,3.252609


Lte's look at the 28th row. The lift of 21.196419 means that the likelihood of buying marshmallows and orange bowl together is 21.196419 times more than the likelihood of just buying orange bowl. The support of 0.02 means that they appear in transactions together in about 2% of all transactions.

Also, there seems to be a lot of cutlery purchased so these purchased cutlery are given as gifts

# 2. France

In [89]:
#5% Support means total 5% of transactions in database follow the rule.

frq_items = apriori(basket_France, min_support = 0.01, use_colnames = True) 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(4 PINK DINNER CANDLE SILVER FLOCK),(IVORY PILLAR CANDLE SILVER FLOCK),0.011561,0.011561,0.011561,1.0,86.500000,0.011427,inf
1,(IVORY PILLAR CANDLE SILVER FLOCK),(4 PINK DINNER CANDLE SILVER FLOCK),0.011561,0.011561,0.011561,1.0,86.500000,0.011427,inf
32,(CHOCOLATE BOX RIBBONS),(ROMANTIC PINKS RIBBONS),0.011561,0.011561,0.011561,1.0,86.500000,0.011427,inf
33,(ROMANTIC PINKS RIBBONS),(CHOCOLATE BOX RIBBONS),0.011561,0.011561,0.011561,1.0,86.500000,0.011427,inf
64,(S/4 PINK FLOWER CANDLES IN BOWL),(SET/4 RED MINI ROSE CANDLE IN BOWL),0.011561,0.011561,0.011561,1.0,86.500000,0.011427,inf
65,(SET/4 RED MINI ROSE CANDLE IN BOWL),(S/4 PINK FLOWER CANDLES IN BOWL),0.011561,0.011561,0.011561,1.0,86.500000,0.011427,inf
17,(BLUE POLKADOT GARDEN PARASOL),(PINK POLKADOT GARDEN PARASOL),0.011561,0.017341,0.011561,1.0,57.666667,0.011360,inf
49,(GREEN 3 PIECE POLKADOT CUTLERY SET),(RED 3 PIECE RETROSPOT CUTLERY SET),0.011561,0.017341,0.011561,1.0,57.666667,0.011360,inf
84,"(GREEN 3 PIECE POLKADOT CUTLERY SET, BLUE 3 PI...",(RED 3 PIECE RETROSPOT CUTLERY SET),0.011561,0.017341,0.011561,1.0,57.666667,0.011360,inf
86,(GREEN 3 PIECE POLKADOT CUTLERY SET),"(RED 3 PIECE RETROSPOT CUTLERY SET, BLUE 3 PIE...",0.011561,0.017341,0.011561,1.0,57.666667,0.011360,inf


Let's look at the 1st row. The lift of 86.5 means that the likelihood of buying 4 pink dinner candle silver flock and ivory pillar candle silve flock together is 86.5 times more than the likelihood of just buying ivory pillar candle. The support of 0.01 means that they appear in transactions together in about 1% of all transactions.

Again, here also there seems to be a lot of cutlery purchased along with chocolates. So , in France mostly cutlery and chocolates are given as gifts.

# 3. Germany

In [90]:
#5% Support means total 5% of transactions in database follow the rule.

frq_items = apriori(basket_Germany, min_support = 0.01, use_colnames = True) 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
64,(SET OF 16 VINTAGE RED CUTLERY),(SET OF 16 VINTAGE SKY BLUE CUTLERY),0.014019,0.018692,0.014019,1.0,53.500000,0.013757,inf
48,(ENAMEL PINK TEA CONTAINER),(ENAMEL PINK COFFEE CONTAINER),0.014019,0.023364,0.014019,1.0,42.800000,0.013691,inf
74,"(CHILDRENS CUTLERY RETROSPOT RED, CHILDRENS CU...",(CHILDRENS CUTLERY POLKADOT BLUE),0.018692,0.046729,0.018692,1.0,21.400000,0.017818,inf
72,(BABY BOOM RIBBONS),"(CHOCOLATE BOX RIBBONS, SCANDINAVIAN REDS RIBB...",0.014019,0.056075,0.014019,1.0,17.833333,0.013233,inf
121,"(URBAN BLACK RIBBONS, ROMANTIC PINKS RIBBONS)","(CHOCOLATE BOX RIBBONS, SCANDINAVIAN REDS RIBB...",0.018692,0.056075,0.018692,1.0,17.833333,0.017643,inf
75,"(CHILDRENS CUTLERY RETROSPOT RED, CHILDRENS CU...",(CHILDRENS CUTLERY POLKADOT PINK),0.018692,0.060748,0.018692,1.0,16.461538,0.017556,inf
81,"(JUMBO BAG RED RETROSPOT, CHILDRENS CUTLERY PO...",(CHILDRENS CUTLERY POLKADOT PINK),0.014019,0.060748,0.014019,1.0,16.461538,0.013167,inf
16,(PINK HAPPY BIRTHDAY BUNTING),(BLUE HAPPY BIRTHDAY BUNTING),0.042056,0.065421,0.042056,1.0,15.285714,0.039305,inf
0,(ASS FLORAL PRINT MULTI SCREWDRIVER),(RED RETROSPOT UMBRELLA),0.014019,0.070093,0.014019,1.0,14.266667,0.013036,inf
105,"(EDWARDIAN PARASOL NATURAL, EDWARDIAN PARASOL ...",(EDWARDIAN PARASOL BLACK),0.028037,0.074766,0.028037,1.0,13.375000,0.025941,inf


Let's look at the 1st row. The lift of 53.5 means that the likelihood of buying red cutlery and blue cutlery together is 53.5 times more than the likelihood of just buying blue cutlery. The support of 0.01 means that they appear in transactions together in about 1% of all transactions.

Here, we have a lot of cutlery, ribbons and buntings that are bought as gifts.

<b>For small datasets:</b>

Source: https://stackoverflow.com/questions/49518187/converting-a-apriori-object-to-a-list-taking-more-time-even-for-small-number-of

In cases where even a small number of transactions is taking a long time to run it's often a matter of too low of a minimum support. When support is very low (near 0) the algorithm is effectively still brute forcing, since it has to look at all possible combinations of items, of every length. This is the equivalent of a mathematical power set, which is exponential. For just 41 items you're actually trying 2^41 -1 possible combinations, which is just over 1.1 TRILLION possibilities.

I recommend starting with a "high" min_support at first (e.g. 0.20) and then working your way down slowly. It's easier to test things that take seconds at first than ones that'll take a long time.